In [158]:
 from julia import Julia
jl = Julia(sysimage = "/home/callum/ASF/Fitting/sys_model.so") #loading sys image

In [28]:
import matplotlib.pyplot as plt

import os
import tempfile
import numpy as np
import scipy as sp


from pyabc import ABCSMC, RV, Distribution, LocalTransition, MedianEpsilon, SingleCoreSampler
from pyabc.visualization import plot_data_callback, plot_kde_2d
from pyabc.populationstrategy import AdaptivePopulationSize
import pyabc
from scipy import stats
import random as rd
from brokenaxes import brokenaxes

In [365]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

In [385]:
plt.style.use('seaborn-colorblind')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [185]:
def read_inputs(path,number):
    
    models = [1,2,3,4]
    
    if number not in models:
        raise Exception("Model number must be 1, 2, 3 or 4") 
        
    weights = np.genfromtxt(path + "/h{}_weights.csv".format(number), delimiter=',')
    params = np.genfromtxt(path + "/h{}_params.csv".format(number), delimiter=',')
    
    nparams = params.shape[1]
    
    p1_dis = stats.gaussian_kde(params[:,0],weights = weights)
    p2_dis = stats.gaussian_kde(params[:,1],weights = weights)
    
    if nparams == 2:
        return p1_dis, p2_dis
    elif nparams == 3:
        p3_dis = stats.gaussian_kde(params[:,2],weights = weights)
        return p1_dis, p2_dis, p3_dis
    

In [196]:
def run_simulations(path,number,n_sims, median = False, distance = False):
    
    posteriors = read_inputs(path,number)
    
    n_params = len(posteriors) #two or three fitted parameters!
    
    store  = np.zeros((n_sims,3))
    
    dd = 0
    n_samples = 100000
    
    dp1 = posteriors[0].resample(n_samples)[0]
    dp2 = posteriors[1].resample(n_samples)[0]

    rd.shuffle(dp1)
    rd.shuffle(dp2)
    
    if n_params == 3:
        dp3 = posteriors[2].resample(n_samples)[0]
        rd.shuffle(dp3)
        
    if median:
        if n_params == 2:
            p = {"p1":np.median(dp1),"p2":np.median(dp2)}
        elif n_params ==3:
            p = {"p1":np.median(dp1),"p2":np.median(dp2), "p3":np.median(dp3)}
            
    for i in range(n_sims):
        
        if not median:
            if n_params == 2:
                p = {"p1":dp1[i],"p2":dp2[i]}
            elif n_params ==3:
                p = {"p1":dp1[i],"p2":dp2[i], "p3":dp3[i]}

        
        
        if number == 1:
            out = model1(p)
            
        elif number == 2:
             out = model2(p)
                
        elif number == 3:
             out = model3(p)
                
        else:
             out = model4(p)

        
        store[i] = out["SS"][0:3]
        dd += out["SS"][3]
    print("Finished")
    if distance:
        dis, mean, c95 = distance_from_mean(store)
        
        return store, dis, mean, c95
    
    else:        
        return store, dd
    
    
        
        

In [7]:
def distance_from_mean(store):
    
    mean_ep = 1.5
    std_ep = 0.604

    mean_pd = 75
    std_pd = 6.08

    mean_mt = 180
    std_mt = 36.475

    
    store_t = (store -[mean_ep, mean_pd, mean_mt]) / ([std_ep, std_pd, std_mt])
    
    d = np.linalg.norm(store_t,axis=1)
    
    mean = np.mean(d)
    c_95 = np.percentile(d,[2.5, 97.5])
    
    print('Mean distance = ', mean)
    print('95% confidence interval =', c_95)
    
    return d, mean, c_95

In [50]:
def distri(data):
    nt =  data.n_populations - 1 
    h_ws = data.get_distribution(t=nt)[1]
    h_p1 = data.get_distribution(t=nt)[0]['p1'].values
    h_p2 = data.get_distribution(t=nt)[0]['p2'].values
    h_p3 = data.get_distribution(t=nt)[0]['p3'].values

    p1_dis = stats.gaussian_kde(h_p1,weights = h_ws)
    p2_dis = stats.gaussian_kde(h_p2,weights = h_ws)
    p3_dis = stats.gaussian_kde(h_p3,weights = h_ws)

    return p1_dis, p2_dis, p3_dis

In [77]:
def run_sim(data,n_sims, modeln):
    p1, p2,p3 = distri(data)
    store  = np.zeros((n_sims,3))
    
    dp1 = p1.resample(n_sims)[0]
    dp2 = p2.resample(n_sims)[0]
    dp3 = p3.resample(n_sims)[0]
    
    for i in range(n_sims):
        
        
        p = {"p1":dp1[i],"p2":dp2[i], "p3":dp3[i]}
        #p = {"p1":np.average(p1_h1, weights=w_h1),"p2":np.average(p2_h1, weights=w_h1)}
        out = modeln(p)
        
        store[i] = out["SS"]
    
    
    store_t = (store - obs["SS"]) / ([std_ep, std_pd, std_mt])
    
    d = np.linalg.norm(store_t,axis=1)
    
    mean = np.mean(d)
    c_95 = np.percentile(d,[2.5, 97.5])
    
    print('Mean distance = ', mean)
    print('95% confidence interval =', c_95)
    
    return store#mean, c_95

In [21]:
mean_ep = 1.5
std_ep = 0.604

mean_pd = 75
std_pd = 6.08

mean_mt = 180
std_mt = 36.475



In [22]:
ode_1 = run_simulations("Posteriors/ODE/", 1, 10000)
ode_2 = run_simulations("Posteriors/ODE/", 2, 10000)
ode_3 = run_simulations("Posteriors/ODE/", 3, 10000)
ode_4 = run_simulations("Posteriors/ODE/", 4, 10000)

Finished
Finished
Finished
Finished


# ODE model

In [10]:
jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/ODE_FIT.jl")')
jl.include('/home/callum/ASF/Fitting/ODE_FIT.jl') #loading files with our model!

<PyCall.jlwrap Main.SIR_ODE>

In [13]:
#Loading the four models!
model1 = jl.SIR_ODE.model_1 #Frequency
model2 = jl.SIR_ODE.model_2 #Pure Density
model3 = jl.SIR_ODE.model_3 #Sigmoid Density
model4 = jl.SIR_ODE.model_4 #Sqrt Density

models  =[model1,model2,model3,model4]

/home/callum/.local/lib/python3.10/site-packages/julia/core.py:709: FutureWarning: Accessing `Julia().<name>` to obtain Julia objects is deprecated.  Use `from julia import Main; Main.<name>` or `jl = Julia(); jl.eval('<name>')`.
  warnings.warn(


In [12]:
#The observations!
obs = jl.SIR_ODE.observation

#Distance function used for fitting
distance = jl.SIR_ODE.distance


In [ ]:
#Fitting two params P1 - transmission co-efficent, P2 - corpse infectivity modifier
parameter_prior = Distribution(p1=RV("uniform", 0.0, 1.0),p2=RV("uniform", 0.0, 1.0))

#sample params to test functions all work and pre-compile
sample_par = {"p1": 0.1,"p2": 0.5} 

n = 4 #number of models we are looking at!
param_list = [parameter_prior]*n

In [ ]:
distance(model1(sample_par),obs)
distance(model2(sample_par),obs)
distance(model3(sample_par),obs)
distance(model4(sample_par),obs);

In [ ]:
abc= ABCSMC(
    models=model4,
    parameter_priors=parameter_prior,
    distance_function=distance,
    population_size =  AdaptivePopulationSize(1000, 0.15,max_population_size = 1000),
    sampler = SingleCoreSampler(),
)



In [ ]:
abc_id = abc.new("sqlite:////tmp/mp.db", obs)


In [ ]:
h4 = abc.run(minimum_epsilon=0.5,max_nr_populations=20)


# Tau Leaping Homogeneous 

In [161]:
jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/TAU_HOMO_FIT.jl")')
jl.include('/home/callum/ASF/Fitting/TAU_HOMO_FIT.jl') #loading files with our model!

<PyCall.jlwrap Main.SIR_TAU_S>

In [162]:
model1 = jl.SIR_TAU_S.model_1 #Frequency
model2 = jl.SIR_TAU_S.model_2 #Pure Density
model3 = jl.SIR_TAU_S.model_3 #Sigmoid Density
model4 = jl.SIR_TAU_S.model_4 #Sqrt Density

#models_h  =[model1_h,model2_h,model3_h,model4_h]

In [ ]:
distance(model1_h (sample_par),obs)
distance(model2_h (sample_par),obs)
distance(model3_h (sample_par),obs)
distance(model4_h (sample_par),obs);

In [ ]:
abc = ABCSMC(
    models=model4_h,
    parameter_priors=parameter_prior,
    distance_function=distance,
    population_size =  AdaptivePopulationSize(1000, 0.15,max_population_size = 1000),
    sampler = SingleCoreSampler(),
)

In [ ]:

abc_id = abc.new("sqlite:////tmp/mjp.db", obs)


In [ ]:

h4_h = abc.run(minimum_epsilon=0.5, max_nr_populations=15)


# Tau Hetro

In [163]:
jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/TAU_HETRO_FIT.jl")')
jl.include('/home/callum/ASF/Fitting/TAU_HETRO_FIT.jl') #loading files with our model!

<PyCall.jlwrap Main.SIR_TAU_M>

In [165]:
model1= jl.SIR_TAU_M.model_1 #Frequency
#model2 = jl.SIR_TAU_M.model_2 #Pure Density
#model3 = jl.SIR_TAU_M.model_3 #Sigmoid Density
model4 = jl.SIR_TAU_M.model_4 #Sqrt Density

In [9]:
#Fitting two params P1 - transmission co-efficent, P2 - corpse infectivity modifier
parameter_prior = Distribution(p1=RV("uniform", 0.0, 1.0),p2=RV("uniform", 0.0, 0.25),p3=RV("uniform", 0.0, 1.0),)

#sample params to test functions all work and pre-compile
sample_par = {"p1": 0.1,"p2": 0.05, "p3": 0.5} 

n = 4 #number of models we are looking at!
param_list = [parameter_prior]*n

In [10]:
distance(model1_c (sample_par),obs)
distance(model2_c (sample_par),obs)
distance(model3_c (sample_par),obs)
distance(model4_c (sample_par),obs);

NameError: name 'distance' is not defined

In [12]:
abc = ABCSMC(
    models=model1_c,
    parameter_priors=parameter_prior,
    distance_function=distance,
    population_size =  AdaptivePopulationSize(100, 0.15,max_population_size = 100),
    sampler = SingleCoreSampler(),
)




In [13]:
abc_id = abc.new("sqlite:////tmp/mjjp.db", obs)


ABC.History INFO: Start <ABCSMC id=1, start_time=2023-03-21 21:08:47>


In [14]:
h1n = abc.run(minimum_epsilon=0.5, max_nr_populations=6)


ABC INFO: Calibration sample t = -1.
ABC INFO: t: 0, eps: 4.98509856e+00.
ABC INFO: Accepted: 100 / 188 = 5.3191e-01, ESS: 1.0000e+02.
ABC.Adaptation INFO: Change nr particles 100 -> 100
ABC INFO: t: 1, eps: 3.53003223e+00.
ABC INFO: Accepted: 100 / 247 = 4.0486e-01, ESS: 8.3813e+01.
ABC.Adaptation INFO: Change nr particles 100 -> 100
ABC INFO: t: 2, eps: 2.50887318e+00.
ABC INFO: Accepted: 100 / 285 = 3.5088e-01, ESS: 6.7366e+01.
ABC.Adaptation INFO: Change nr particles 100 -> 100
ABC INFO: t: 3, eps: 1.79454663e+00.
ABC INFO: Accepted: 100 / 350 = 2.8571e-01, ESS: 7.3575e+01.
ABC.Adaptation INFO: Change nr particles 100 -> 100
ABC INFO: t: 4, eps: 1.22487862e+00.
ABC INFO: Accepted: 100 / 750 = 1.3333e-01, ESS: 8.0965e+01.
ABC.Adaptation INFO: Change nr particles 100 -> 100
ABC INFO: t: 5, eps: 9.27977813e-01.
ABC INFO: Accepted: 100 / 1333 = 7.5019e-02, ESS: 8.6901e+01.
ABC.Adaptation INFO: Change nr particles 100 -> 100
ABC INFO: Stop: Maximum number of generations.
ABC.History INF